In [1]:
# Required Imports
from netCDF4 import Dataset as netcdf_dataset
from datetime import datetime, timedelta
from matplotlib.offsetbox import AnchoredText

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from siphon.catalog import TDSCatalog
from xarray.backends import NetCDF4DataStore

import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
import cmaps

import sys
import ipywidgets as widgets

import warnings
warnings.filterwarnings('ignore') # hides no contour warnings

In [2]:
# Import regions functions from sstAnomsFunctions.py
from sstAnomsFunctions import North_Atlantic, Gulf_of_Mexico, World, Arctic
from sstAnomsFunctions import Antarctic, Equatorial_Pacific, Tropical_Atlantic

# Select Region

In [3]:
RegionValue = widgets.Dropdown(
    options=[('Antarctic', 1), ('Arctic', 2), ('Equatorial_Pacific', 3), ('Gulf_of_Mexico', 4),
             ('North_Atlantic', 5), ('Tropical_Atlantic', 6), ('World', 7)],
    value=1,
    description='Select Region:',
)
RegionValue

Dropdown(description='Select Region:', options=(('Antarctic', 1), ('Arctic', 2), ('Equatorial_Pacific', 3), ('…

# Select Days Back start and end
### Yesterday for starttime = 1 if time after 12Z or before 0Z
### Yesterday for starttime = 2 if time after 0z or before 12Z

In [4]:
daysBackStartValue = widgets.BoundedIntText(
    value=1,
    min=1,
    max=370,
    step=1,
    description='Start',
    disabled=False
)
daysBackStartValue

BoundedIntText(value=1, description='Start', max=370, min=1)

In [5]:
daysBackEndValue = widgets.BoundedIntText(
    value=2,
    min=2,
    max=740,
    step=1,
    description='End',
    disabled=False
)
daysBackEndValue

BoundedIntText(value=2, description='End', max=740, min=2)

In [19]:
# Get region from widget selection
region = RegionValue.label

# Get days from widget selection
daysBackStart = daysBackStartValue.value
daysBackEnd = daysBackEndValue.value

# Set text for the different regions
txtWorld = "World: "
txtArctic = "Arctic: "
txtAntarctic = "Antarctic: "
txtNH = "NH: "
txtSH = "SH: "
txtNino34 = "Niño 3.4: "
txtTropics = "Tropics: "
txtNPac = "N. Pacific: "
txtNAtl = "N. Atlantic: "

In [20]:
### GENERATE PLOTS ###
for i in range(daysBackStart, daysBackEnd,1):
    today = datetime.utcnow()
    date = datetime(today.year, today.month, today.day, 12) - timedelta(days=i)
    
    if i <= 14: # 14 if time is after 12Z and before 0Z UTC; 15 if after 0Z and before 12Z
    # Preliminary datasets
        base_url = 'https://www.ncei.noaa.gov/thredds/catalog/OisstBase/NetCDF/V2.1/AVHRR/'
        cat = TDSCatalog(f'{base_url}{date:%Y%m}/catalog.xml')
        ncss = cat.datasets[f'oisst-avhrr-v02r01.{date:%Y%m%d}_preliminary.nc'].subset()
    else:
    # Official datasets
        base_url = 'https://www.ncei.noaa.gov/thredds/catalog/OisstBase/NetCDF/V2.1/AVHRR/'
        cat = TDSCatalog(f'{base_url}{date:%Y%m}/catalog.xml')
        ncss = cat.datasets[f'oisst-avhrr-v02r01.{date:%Y%m%d}.nc'].subset()
  
    query = ncss.query()
    query.time(date)
    
    ### DEFAULT SETTINGS ###
    # Changed as necessary for certain regions
    # IMPORTANT: Update if you make changes to the sstAnomsFunctions.py file #
    projection = ccrs.PlateCarree()
    cenLon = 0
    contouring = False
    if region == 'North_Atlantic':
        regionName, savePathAnoms, fileNameAnoms, north, south, east, west, extent = North_Atlantic()
        contouring = True
    elif region == 'Gulf_of_Mexico':
        regionName, savePathAnoms, fileNameAnoms, north, south, east, west, extent = Gulf_of_Mexico()
        plt.rcParams['axes.facecolor'] = 'black'
    elif region == 'World':
        regionName, savePathAnoms, fileNameAnoms, north, south, east, west, extent = World()
    elif region == 'Arctic':
        regionName, savePathAnoms, fileNameAnoms, north, south, east, west, extent = Arctic()
        plt.rcParams['axes.facecolor'] = 'black'
        projection = ccrs.NorthPolarStereo()
        contouring = True
    elif region == 'Antarctic':
        regionName, savePathAnoms, fileNameAnoms, north, south, east, west, extent = Antarctic()
        projection = ccrs.SouthPolarStereo()
    elif region == 'Equatorial_Pacific':
        regionName, savePathAnoms, fileNameAnoms, north, south, east, west, extent = Equatorial_Pacific()
        projection = ccrs.PlateCarree(central_longitude=-180)
        cenLon = -180
    elif region == 'Tropical_Atlantic':
        regionName, savePathAnoms, fileNameAnoms, north, south, east, west, extent = Tropical_Atlantic()
    else:
        print('Error: Please select a valid region from above and try again.')
        sys.exit()
    ### END DEFAULT SETTINGS ###

    # Get data
    query.lonlat_box(north=north, south=south, east=east, west=west)
    query.accept('netcdf')
    query.variables('anom')
    
    data = ncss.get_data(query)
    ds = xr.open_dataset(NetCDF4DataStore(data))

    # Set variable names from dataset
    sstAnoms = ds.variables['anom'][0,0,:,:]
    lats = ds.variables['lat'][:]
    lons = ds.variables['lon'][:]
    
    # SST Anomaly Calculations (not 100% accurate as compared to NASA/NOAA calculations)
    sstMeanAnom = sstAnoms.mean().values
    sstMeanAnomArctic = sstAnoms.where(lats >= 65).mean().values
    sstMeanAnomAntarctic = sstAnoms.where(lats <= 65).mean().values
    sstMeanAnomNH = sstAnoms.where(lats >= 0).mean().values
    sstMeanAnomSH = sstAnoms.where(lats <= 0).mean().values
    
    # Equatorial Eastern Pacific Mean (Niño 3.4 region)
    idx_lat_nino34  = (lats>=-5.0) * (lats<=5.0)
    idx_lon_nino34  = (lons>=190) * (lons<=240.0)
    sstMeanAnomNino34 = ds.variables['anom'].load()
    sstMeanAnomNino34 = sstMeanAnomNino34[0,0,idx_lat_nino34,:][:,idx_lon_nino34].mean().values
    
    # Tropics Mean
    idx_lat_tropics  = (lats>=-25.0) * (lats<=25.0)
    sstMeanAnomTropics  = ds.variables['anom'][0,0,idx_lat_tropics,:][:,:].mean().values

    # North Pacific Mean
    idx_lat_npac  = (lats>=0.0) * (lats<=60.0)
    idx_lon_npac  = (lons>=130) * (lons<=260.0)
    sstMeanAnomNPac  = ds.variables['anom'][0,0,idx_lat_npac,:][:,idx_lon_npac].mean().values
    
    # North Atlantic Mean
    idx_lat_natl  = (lats>=5.0) * (lats<=65.0)
    idx_lon_natl  = (lons>=280) * (lons<=360.0)
    sstMeanAnomNAtl  = ds.variables['anom'][0,0,idx_lat_natl,:][:,idx_lon_natl].mean().values
    
    # Set symbol and color for mean anomaly
    if sstMeanAnom > 0:
        symbol = '+'
        color = 'red'
    else:
        symbol = ''
        color = 'blue'
        
    if sstMeanAnomArctic > 0:
        symbolArctic = '+'
        colorArctic = 'red'
    else:
        symbolArctic = ''
        colorArctic = 'blue'
        
    if sstMeanAnomAntarctic > 0:
        symbolAntarctic = '+'
        colorAntarctic = 'red'
    else:
        symbolAntarctic = ''
        colorAntarctic = 'blue'
        
    if sstMeanAnomNH > 0:
        symbolNH = '+'
        colorNH = 'red'
    else:
        symbolNH = ''
        colorNH = 'blue'
    
    if sstMeanAnomSH > 0:
        symbolSH = '+'
        colorSH = 'red'
    else:
        symbolSH = ''
        colorSH = 'blue'
        
    if sstMeanAnomNino34 > 0:
        symbolNino34 = '+'
        colorNino34 = 'red'
    else:
        symbolNino34 = ''
        colorNino34 = 'blue'
    
    if sstMeanAnomTropics > 0:
        symbolTropics = '+'
        colorTropics = 'red'
    else:
        symbolTropics = ''
        colorTropics = 'blue'
    
    if sstMeanAnomNPac > 0:
        symbolNPac = '+'
        colorNPac = 'red'
    else:
        symbolNPac = '-'
        colorNPac = 'blue'
        
    if sstMeanAnomNAtl > 0:
        symbolNAtl = '+'
        colorNAtl = 'red'
    else:
        symbolNAtl = ''
        colorNAtl = 'blue'
    
    # Set up figure and axes
    fig = plt.figure(figsize=(15,10), facecolor='black')
    ax = fig.add_subplot(111, projection=projection)
    ax.set_extent(extent, crs=ccrs.PlateCarree(central_longitude=cenLon))
    
    # Define colormap, normalization, and levels
    cmap = cmaps.cmocean_balance
    min_plot_value = -5
    max_plot_value = +5
    contour_level = 0.1
    clevs = np.arange(min_plot_value,max_plot_value,contour_level)

    # Plot data and specified contour levels
    sst_contour = ax.contourf(lons, lats, sstAnoms, clevs, cmap=cmap,
                              transform=ccrs.PlateCarree(), extend='both')
    
    if contouring == True:
        sstAnom5 = ax.contour(lons, lats, sstAnoms, levels=[5], colors='magenta', 
                              linestyles=('--'), linewidths=(1), transform=ccrs.PlateCarree())
        
        sstAnomMin5 = ax.contour(lons,lats,sstAnoms, levels=[-5], colors='white',
                                 linestyles=('--'), linewidths=(1), transform=ccrs.PlateCarree())
        
        plt.clabel(sstAnom5, fontsize=10, inline=1, fmt='%1.0f')
        plt.clabel(sstAnomMin5, fontsize=10, inline=1, fmt='%1.0f')
    else:
        pass

    # add map features
    ax.add_feature(cfeature.LAND, color='lightgray')
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=1)
    ax.add_feature(cfeature.BORDERS.with_scale('50m'), linewidth=1)
    ax.add_feature(cfeature.STATES, linewidth=0.8)

    # colobar parameters
    cb_ax = inset_axes(ax, width='2%', height='100%', loc='lower right',
                       bbox_to_anchor=(0,0,1.024,1), bbox_transform=ax.transAxes, borderpad=0)

    cbar = plt.colorbar(sst_contour, cax=cb_ax)
    cbar.set_label('Temperature Anomalies (°C)', color='white', rotation=270, 
                   labelpad=18, weight='bold', size=13)
    cbar.ax.tick_params(labelsize=12, colors='white')

    # Set annotation for creator and data source
    at = AnchoredText(
            'Created by: Christopher Turnbull\n'+
            'Data: NOAA NCEI OisstBase AVHRR',
            prop=dict(size=12), frameon=True, loc='lower left')

    at.patch.set_boxstyle('round, pad=0., rounding_size=0.2')
    at.patch.set_facecolor('white')
    ax.add_artist(at)

    # Set Titles
    ax.set_title(regionName+' - Sea Surface Temperature Anomalies (°C)', loc='left', weight='bold', color='white')
    ax.set_title('Valid: '+date.strftime('%Hz %a %b %d %Y'), loc='right', weight='bold', color='white')
    
    if region == 'World':
        plt.figtext(0.25, 0.16, txtWorld+symbol+str(np.round(sstMeanAnom,2))+'°C', wrap=True, 
                    horizontalalignment='center', fontsize=12, color=color)
    
        plt.figtext(0.40, 0.16, txtArctic+symbolArctic+str(np.round(sstMeanAnomArctic,2))+'°C', wrap=True,
                    horizontalalignment='center', fontsize=12, color=colorArctic)
    
        plt.figtext(0.55, 0.16, txtAntarctic+symbolAntarctic+str(np.round(sstMeanAnomAntarctic,2))+'°C', wrap=True,
                    horizontalalignment='center', fontsize=12, color=colorAntarctic)

        plt.figtext(0.70, 0.16, txtNH+symbolNH+str(np.round(sstMeanAnomNH,2))+'°C', wrap=True,
                    horizontalalignment='center', fontsize=12, color=colorNH)

        plt.figtext(0.85, 0.16, txtSH+symbolSH+str(np.round(sstMeanAnomSH,2))+'°C', wrap=True,
                    horizontalalignment='center', fontsize=12, color=colorSH)

        plt.figtext(0.25, 0.11, txtNino34+symbolNino34+str(np.round(sstMeanAnomNino34,2))+'°C', wrap=True,
                    horizontalalignment='center', fontsize=12, color=colorNino34)

        plt.figtext(0.40, 0.11, txtTropics+symbolTropics+str(np.round(sstMeanAnomTropics,2))+'°C', wrap=True,
                    horizontalalignment='center', fontsize=12, color=colorTropics)


        plt.figtext(0.55, 0.11, txtNPac+symbolNPac+str(np.round(sstMeanAnomNPac,2))+'°C', wrap=True,
                    horizontalalignment='center', fontsize=12, color=colorNPac)

        plt.figtext(0.70, 0.11, txtNAtl+symbolNAtl+str(np.round(sstMeanAnomNAtl,2))+'°C', wrap=True,
                    horizontalalignment='center', fontsize=12, color=colorNAtl)
    elif region == 'North_Atlantic':
        ax.set_title(symbolNAtl+str(np.round(sstMeanAnomNAtl,2))+'°C', color=colorNAtl, loc='center')
    elif region == 'Arctic':
        plt.figtext(0.78,0.89, symbolArctic+str(np.round(sstMeanAnomArctic,2))+'°C', color=colorArctic)
        #ax.set_title('\n'+symbolArctic+str(np.round(sstMeanAnomArctic,2))+'°C', color=colorArctic, loc='left')
    elif region == 'Equatorial_Pacific':
        plt.figtext(0.60,0.76, symbolNino34+str(np.round(sstMeanAnomNino34,2))+'°C', color=colorNino34)
    
    # Save figure and close plot window
    # Modify path as needed
    plt.savefig('images/sstAnoms/'+savePathAnoms+fileNameAnoms+date.strftime('%Y%m%d-%Hz')+'.png',dpi=200,
                bbox_inches='tight')
    
    ### Don't use plt.show()! ###
    plt.close()